In [77]:
#before using this you need to make a link-breaked version of the roads in yoru database with the osm2pgrourting tool. 
#before use it requires a security setup: (for security reasons it wqill only run on a database which has a password setup):
#$ sudo -u postgres psql
# \password
# (enter 'postgres' for the password twice)
# Ctrl-D (to exit psql)
#Then run the link-breaking with:
#$ osm2pgrouting -f data/dcc.osm -d mydatabasename -W postgres
#you also need to do
# $ psql -d mydatabasename
# CREATE EXTENSION pgrouting
#in psql, to enable postgres routing extension.

##This code has cleaned detection table with exceptions and it can find the OD flow-OD distance relation ##Task 1

import psycopg2
import pandas as pd
import geopandas as gpd
import pyproj
import os,re,datetime
from matplotlib.pyplot import *
con = psycopg2.connect(database='mydatabasename', user='root')
cur = con.cursor()
wgs84  = pyproj.Proj(init='epsg:4326')  #WGS84
bng    = pyproj.Proj(init='epsg:27700') #british national grid
import pandas.io.sql as psql # To read sql files
#import random
#from random import randint
print ("import complete")

import complete


In [4]:
#Importing Geometries

def importRoads():     #(data has come from openstreetmap, then ogr2ogr )
    print("importing roads...")
    sql = "DROP TABLE IF EXISTS Road;"
    cur.execute(sql)
    sql = "CREATE TABLE Road (name text, geom geometry, highway text);"
    cur.execute(sql)
    fn_osm_shp = "/headless/data/dcc.osm.shp/lines.shp"
    df_roads = gpd.GeoDataFrame.from_file(fn_osm_shp)
    df_roads = df_roads.to_crs({'init': 'epsg:27700'})
    for index, row in df_roads.iterrows():
        sql="INSERT INTO Road VALUES ('%s', '%s', '%s');"\
        %(row.name, row.geometry, row.highway )
        cur.execute(sql)
    con.commit()
    print ("Roads imported")

importRoads()

importing roads...
Roads imported


In [5]:
##Creating list of BT sites table
def importBluetoothSites():
    print("importing sites...")
    sql = "DROP TABLE IF EXISTS BluetoothSite;"
    cur.execute(sql)
    sql = "CREATE TABLE BluetoothSite ( id serial PRIMARY KEY, siteID text,\
    geom geometry);"
    cur.execute(sql)
    con.commit()    
    fn_sites = "/headless/data/dcc/web_bluetooth_sites.csv"
    df_sites = pd.read_csv(fn_sites, header=1)   #dataframe. header is which row to use for the field names.
    for i in range(0, df_sites.shape[0]):      #munging to extract the coordinates - the arrive in National Grid
        locationstr = str(df_sites.iloc[i]['Grid'])
        bng_east  = locationstr[0:6]
        bng_north = locationstr[6:12]
        sql = "INSERT INTO BluetoothSite (siteID, geom) \
        VALUES ('%s', 'POINT(%s %s)');"%(df_sites.iloc[i]['Site ID'], \
        bng_east, bng_north )
        cur.execute(sql)
    con.commit()        
    print ("BT Sites imported")
importBluetoothSites()

importing sites...
BT Sites imported


In [6]:
##Importing BTsite detection site CSVs and cleaning the repetitive counts, 
##with some exceptions of alternate detections


def importDetections():
    print("importing detections...")
    sql = "DROP TABLE IF EXISTS detection_clean;"
    cur.execute(sql)    
    sql = "CREATE TABLE detection_clean ( id serial, siteID text, mac text, \
    timestamp timestamp );"
    cur.execute(sql)    
    dir_detections = "/headless/data/dcc/bluetooth/"
    for fn in sorted(os.listdir(dir_detections)):  #importing all BT sensor files
        print("processing file: " +fn)
        m = re.match("vdFeb14_(.+).csv", fn)  #extracting CSVs matching the file names
        if m is None:  #if there was no regex match then continue
            continue   
        siteID = m.groups()[0]
        fn_detections = dir_detections+fn
        #dataframe.header is which row to use for field names-fn
        df_detections = pd.read_csv(fn_detections, header=0)
        prev_ts = "" #empty string for timestamp
        prev_mac = ""#empty string for mac
        for i in range(0, df_detections.shape[0]): 
            #here we use Python's DateTime library to store times properly
            datetime_text = df_detections.iloc[i]['Unnamed: 0']
            #proper Python datetime  
            dt = datetime.datetime.strptime(datetime_text, "%d/%m/%Y %H:%M:%S") 
            
            if prev_ts == "":
                prev_ts = dt
                prev_mac = df_detections.iloc[i]['Number Plate']
                #continue
            else:
                cur_ts = dt
                cur_mac = df_detections.iloc[i]['Number Plate']
                #print (cur_ts)
                diff = cur_ts - prev_ts #difference in timestamps
                #print (cur_ts, prev_ts, cur_mac, prev_mac, diff)
                if cur_mac == prev_mac and diff.seconds<60:
                    continue
                else:
                    prev_mac = df_detections.iloc[i]['Number Plate']
                    prev_ts = dt
                    
            sql = "INSERT INTO detection_clean (siteID, timestamp, mac) VALUES ('%s', '%s', '%s');"\
            %(siteID, dt, df_detections.iloc[i]['Number Plate'])
            cur.execute(sql)
            #print(siteID, dt, df_detections.iloc[i]['Number Plate'])
        
        table = psql.read_sql("SELECT * FROM detection_clean", con)
        # print(table)
        print ("Length of  table: ", len(table), df_detections.shape[0])
    con.commit()
    
    print("Cleaning done")
    
importDetections()



importing detections...
processing file: vdFeb14_MAC000010100.csv
Length of  table:  2955 4521
processing file: vdFeb14_MAC000010101.csv
Length of  table:  7098 5946
processing file: vdFeb14_MAC000010102.csv
Length of  table:  10885 5598
processing file: vdFeb14_MAC000010103.csv
Length of  table:  10960 87
processing file: vdFeb14_MAC000010104.csv
Length of  table:  17865 9356
processing file: vdFeb14_MAC000010105.csv
Length of  table:  21017 4763
processing file: vdFeb14_MAC000010106.csv
Length of  table:  24635 5634
processing file: vdFeb14_MAC000010107.csv
Length of  table:  28373 5664
processing file: vdFeb14_MAC000010108.csv
Length of  table:  36727 10728
processing file: vdFeb14_MAC000010109.csv
Length of  table:  44144 9882
processing file: vdFeb14_MAC000010110.csv
Length of  table:  47665 5256
processing file: vdFeb14_MAC000010111.csv
Length of  table:  51037 5398
processing file: vdFeb14_MAC000010112.csv
Length of  table:  55847 7169
processing file: vdFeb14_MAC000010113.csv
L

In [7]:
def plotRoads():   
    print("plotting roads...")
    sql = "SELECT * FROM Road;"
    df_roads = gpd.GeoDataFrame.from_postgis(sql,con,geom_col='geom') #
    for index, row in df_roads.iterrows():
        (xs,ys) = row['geom'].coords.xy
        color='y'
        #road colour by type
        if row['highway']=="motorway":
            color = 'b'
        if row['highway']=="trunk":
            color = 'g'
        #if not color=='y':  #only plot major roads
        plot(xs, ys, color)
    print("Roads plotted")
plotRoads()

def plotBluetoothSites():    
    sql = "SELECT siteID, geom FROM BluetoothSite;"
    df_sites = gpd.GeoDataFrame.from_postgis(sql,con,geom_col='geom') #
    for index, row in df_sites.iterrows():
        (xs,ys) = row['geom'].coords.xy
        plot(xs, ys, 'bo')
    print ("BT sites plotted")  
plotBluetoothSites()

plotting roads...
Roads plotted
BT sites plotted


In [78]:

def createTotalRoute(con,cur):
    sql = "DROP TABLE IF EXISTS TotalRoute;"
    cur.execute(sql)          
    sql="CREATE TABLE TotalRoute( \
        emp_no SERIAL PRIMARY KEY, \
        ODrouteID text, \
        timestamp timestamp, \
        winlenseconds integer, \
        count integer, OriginSiteID text,\
        DestSiteID text);"
    cur.execute(sql)          
    con.commit() 

createTotalRoute(con,cur)

def makeTotalRoutes(con,cur):
    print("define Total measureable route from origin sensor to destination sensor")

    ODRoutes = {"MeasureableRouteID":["MAC000010119>MAC000010130",\
                                     "MAC000010102>MAC000010104",\
                                     "MAC000010121>MAC000010124",\
                                     "MAC000010101>MAC000010119",\
                                     "MAC000010123>MAC000010120"],\
               "OriginSiteID":["MAC000010119","MAC000010102",\
                               "MAC000010121",\
                               "MAC000010101","MAC000010123"],

                "DestSiteID":["MAC000010130","MAC000010104",\
                              "MAC000010124",\
                              "MAC000010119","MAC000010120"]}
    df=pd.DataFrame.from_dict(ODRoutes)
    for i in range(0,df.shape[0]):    #each route
        routeID = df['MeasureableRouteID'][i]
        oSiteID = df['OriginSiteID'][i]
        dSiteID = df['DestSiteID'][i]
        #MAC matching
        sql = "SELECT d.siteID AS dSiteID,  d.mac as dmac, d.timestamp as dtimestamp,\
        o.siteID AS oSiteID,  o.mac as omac, o.timestamp as otimestamp  \
        FROM detection_clean AS d,detection_clean AS o  \
        WHERE d.timestamp>o.timestamp\
        AND o.mac=d.mac  AND o.siteID='%s'\
        AND d.siteID='%s'"%(oSiteID, dSiteID)

        df_matches = pd.read_sql_query(sql,con)
        count = df_matches.shape[0]  #count number of bluetooth matches
        #these two variables allow us to compute flows for differnt time windows. Here we just take one whole day.
        winlenseconds = 99999999.9
        timestamp = "2015-02-14 09:00:00"
        sql = "INSERT INTO TotalRoute (ODrouteID, timestamp, winlenseconds, count,\
        OriginSiteID, DestSiteID)\
        VALUES ('%s', '%s', %f, %i,'%s','%s')"%(routeID,timestamp,\
        winlenseconds, count, oSiteID, dSiteID)
        cur.execute(sql)
        
    con.commit()  
    print ("Total routes created")
makeTotalRoutes(con,cur)


def createODRoute(con,cur):
    sql = "DROP TABLE IF EXISTS ODRoute;"
    cur.execute(sql)          
    sql="CREATE TABLE ODRoute ( \
            emp_no SERIAL PRIMARY KEY, \
            ODrouteID text, \
            timestamp timestamp, \
            winlenseconds integer, \
            count integer, OriginSiteID text, MidSiteID text, \
            DestSiteID text);"
    cur.execute(sql)          
    con.commit() 

createODRoute(con,cur)

def makeODRoutes(con,cur):
    print("define OD measureable route from each sensor to each other")
    #sql = "SELECT * FROM MeasureableRoute;"
    #df_ODRoutes = pd.read_sql_query(sql,con)
    Routes = {"MeasureableRouteID":["MAC000010119>MAC000010104>MAC000010130",\
                                     "MAC000010119>MAC000010109>MAC000010130",\
                                     "MAC000010102>MAC000010119>MAC000010104",\
                                     "MAC000010102>MAC000010118>MAC000010104",\
                                     "MAC000010121>MAC000010113>MAC000010124",\
                                     "MAC000010121>MAC000010112>MAC000010124",\
                                     "MAC000010101>MAC000010104>MAC000010119",\
                                     "MAC000010101>MAC000010118>MAC000010119",\
                                     "MAC000010123>MAC000010114>MAC000010120",\
                                     "MAC000010123>MAC000010112>MAC000010120"],\
               "OriginSiteID":["MAC000010119","MAC000010119","MAC000010102",\
                               "MAC000010102","MAC000010121","MAC000010121",\
                               "MAC000010101","MAC000010101","MAC000010123",\
                               "MAC000010123"],
                "MidSiteID": ["MAC000010104","MAC000010109","MAC000010119",\
                              "MAC000010118","MAC000010113","MAC000010112",\
                              "MAC000010104","MAC000010118","MAC000010114",\
                              "MAC000010112"],
                "DestSiteID":["MAC000010130","MAC000010130","MAC000010104",\
                              "MAC000010104","MAC000010124","MAC000010124",\
                              "MAC000010119","MAC000010119","MAC000010120",\
                              "MAC000010120"]}
    df=pd.DataFrame.from_dict(Routes)
    #print(df.shape)
    #print(df.shape[0])
    #print("asdf")
# or enter for i in range(0,df.shape[0])    
    for i in range(0,df.shape[0]):    #each route
        routeID = df['MeasureableRouteID'][i]
        oSiteID = df['OriginSiteID'][i]
        mSiteID = df['MidSiteID'][i]
        dSiteID = df['DestSiteID'][i]
        #MAC matching
        sql = "SELECT d.siteID AS dSiteID,  d.mac as dmac, d.timestamp as dtimestamp  ,\
        m.siteID AS mSiteID,  m.mac as mmac, m.timestamp as mtimestamp ,\
        o.siteID AS oSiteID,  o.mac as omac, o.timestamp as otimestamp  \
        FROM detection_clean AS d,detection_clean AS m, detection_clean AS o  \
        WHERE d.timestamp>m.timestamp AND m.timestamp>o.timestamp\
        AND o.mac=m.mac AND m.mac=d.mac  AND o.siteID='%s'AND m.siteID='%s'\
        AND d.siteID='%s'"%(oSiteID, mSiteID, dSiteID)
        #print(sql)
        df_matches = pd.read_sql_query(sql,con)
        count = df_matches.shape[0]  #count number of bluetooth matches
        #these two variables allow us to compute flows for differnt time windows. Here we just take one whole day.
        winlenseconds = 99999999.9
        timestamp = "2015-02-14 09:00:00"
        sql = "INSERT INTO ODRoute (ODrouteID, timestamp, winlenseconds, count,\
        OriginSiteID, MidSiteID, DestSiteID)\
        VALUES ('%s', '%s', %f, %i,'%s','%s','%s')"%(routeID,timestamp,\
        winlenseconds, count, oSiteID, mSiteID, dSiteID)
        cur.execute(sql)
        
    con.commit()  
    print ("OD routes created")
makeODRoutes(con,cur)


define Total measureable route from origin sensor to destination sensor
Total routes created
define OD measureable route from each sensor to each other
OD routes created


In [79]:


def createSensorDistances(con,cur):
    sql = "DROP TABLE IF EXISTS SensorDistances;"
    cur.execute(sql)          
    sql = "CREATE TABLE SensorDistances ( \
            emp_no SERIAL PRIMARY KEY, \
            ODRouteID text, gid_link text, \
            om_length integer, md_length integer,\
            total_length integer\
            );"
    cur.execute(sql) 
    sql = "DROP TABLE IF EXISTS ShortestRoute;"
    cur.execute(sql)        
    sql = "CREATE TABLE ShortestRoute ( \
            emp_no SERIAL PRIMARY KEY, \
            ODrouteID text, \
            om_length integer, \
            md_length integer, \
            total_length integer \
            );"
    cur.execute(sql)      
createSensorDistances(con,cur)

def SensorDistances(con,cur):
    print ("computing lengths ..")
    sql = "SELECT ODRouteID OriginSiteID, MidSiteID, \
            DestSiteID from ODRoute"
    sql = "SELECT ODRouteID, OriginSiteID, MidSiteID, \
            DestSiteID, ST_X(orig.geom) AS ox, ST_Y(orig.geom) \
            AS oy, ST_X(mid.geom) AS mx, ST_Y(mid.geom) AS my, ST_X(dest.geom) AS dx,\
            ST_Y(dest.geom) AS dy from ODRoute, \
            BluetoothSite AS orig, BluetoothSite AS mid, BluetoothSite AS dest \
            WHERE OriginSiteID=orig.siteID AND MidSiteID=mid.siteID AND \
            DestSiteID=dest.siteID;"
    #Read bulk of db in psql and enter into DF
    df_dist = pd.read_sql_query(sql,con)
    print (df_dist)
    N = df_dist.shape[:]   
    #N = len(df_dist)
    print (N, "shape of DF")
# for i in range(0, df_dist) or  i in range (10,7) - Error fixed
    for i in range (0, df_dist.shape[0]): 
        OMDrouteID = df_dist['odrouteid'][i]
        origsiteID = df_dist['originsiteid'][i]
        midsiteID  = df_dist['midsiteid'][i]
        destsiteID = df_dist['destsiteid'][i]
   # for i in range(df_dist)
        o_easting  = df_dist['ox'][i]#link-broken table is stored as latlon so convert on the fly
        #print (o_easting)
        #print(df_dist['ox'][i])
        #print(df_dist['ox'])
        o_northing = df_dist['oy'][i]
        m_easting  = df_dist['mx'][i]
        m_northing = df_dist['my'][i]
        d_easting  = df_dist['dx'][i]
        d_northing = df_dist['dy'][i]
        #print(type(o_easting))
        (o_lon,o_lat) = pyproj.transform(bng, wgs84, o_easting, o_northing) #project uses nonISO lonlat convention
        (m_lon,m_lat) = pyproj.transform(bng, wgs84, m_easting, m_northing)
        (d_lon,d_lat) = pyproj.transform(bng, wgs84, d_easting, d_northing) #project uses nonISO lonlat convention 

    #Origin gid
        sql = "SELECT id,ST_Distance(ways_vertices_pgr.the_geom,\
        ST_SetSRID(ST_MakePoint(%f, %f),4326)) FROM ways_vertices_pgr \
        ORDER BY st_distance ASC LIMIT 1;"%(o_lon,o_lat)    #4326=SRID code for WGS84
        df_on = pd.read_sql_query(sql,con)
        o_vertex_gid = df_on['id'][0] #get the vertexID of its source
        print ("Origin GIDs selected")
    #Mid gid
        sql = "SELECT id,ST_Distance(ways_vertices_pgr.the_geom,\
        ST_SetSRID(ST_MakePoint(%f, %f),4326)) FROM ways_vertices_pgr \
        ORDER BY st_distance ASC LIMIT 1;"%(m_lon,m_lat)    #4326=SRID code for WGS84
        df_mn = pd.read_sql_query(sql,con)
        m_vertex_gid = df_mn['id'][0] #get the vertexID of its source
        print ("Mid GIDs selected")
    
    #Dest gid
        sql = "SELECT id,ST_Distance(ways_vertices_pgr.the_geom,\
        ST_SetSRID(ST_MakePoint(%f, %f),4326)) FROM ways_vertices_pgr \
        ORDER BY st_distance ASC LIMIT 1;"%(d_lon,d_lat)    #4326=SRID code for WGS84
        df_dn = pd.read_sql_query(sql,con)
        d_vertex_gid = df_dn['id'][0] #get the vertexID of its source
        print ("Dest GIDs selected")
        
        #pgr_dijkstra(o_vertex_gid, m_vertex_gid, d_vertex_gid)
        sql = "SELECT * FROM pgr_dijkstra('SELECT gid AS id, source, target, \
        length_m AS cost FROM ways', %d,%d, directed := false), \
        ways  WHERE ways.gid=pgr_dijkstra.edge;"%(o_vertex_gid, m_vertex_gid)
        print ("Shortest path between origin and mid sensors found")
        df_route_1 = gpd.GeoDataFrame.from_postgis(sql, con, geom_col='the_geom')
        OM_route   = pd.read_sql_query(sql,con)
        #print = (OM_route)
        #print ("Origin-mid shortest route")
        # om_dist = 
        
        
        sql = "SELECT * FROM pgr_dijkstra('SELECT gid AS id, source, target, \
        length_m AS cost FROM ways', %d,%d, directed := false), \
        ways  WHERE ways.gid=pgr_dijkstra.edge;"%(m_vertex_gid, d_vertex_gid)
        print ("Shortest path between mid and destination sensors found")
        df_route_2 = gpd.GeoDataFrame.from_postgis(sql, con, geom_col='the_geom')
        MD_route   = pd.read_sql_query(sql,con)

        for i in range(0,df_route_1.shape[0]):
            rc = df_route_1.iloc[i]  #route component
            #route = df_dist.iloc[i]
            sql = "INSERT INTO SensorDistances(ODrouteID, gid_link, om_length) \
            VALUES ('%s', %i, %i);" %(OMDrouteID, rc['gid'],rc['length_m'])
            print(sql)
            cur.execute(sql)
        for i in range(0,df_route_2.shape[0]):
            rc = df_route_2.iloc[i]  #route component
            #route = df_dist.iloc[i]
            sql = "INSERT INTO SensorDistances(ODrouteID, gid_link, md_length) \
            VALUES ('%s', %i, %i);" %(OMDrouteID, rc['gid'],rc['length_m'])
            #sql = "INSERT INTO SensorDistances(ODrouteID,gid_link) values('%s', %d);"\
            #%(OMDrouteID['odrouteid'],rc['gid'])
            print(sql)
            cur.execute(sql)
        sql = "SELECT odrouteid, sum(om_length) as om, sum(md_length) as md from sensordistances\
        group by odrouteid"
        df_sum = pd.read_sql_query(sql,con)
        #df_sum1= df_sum.drop_duplicates()
        for i in range(0,df_sum.shape[0]):
            rl = df_sum.iloc[i]  
            sql = "INSERT INTO ShortestRoute(ODrouteID, om_length, md_length, total_length) \
            VALUES ('%s', %i, %i, %i);" %(rl['odrouteid'], rl['om'],rl['md'], rl['om']+rl['md'])
            print(sql)
            cur.execute(sql)
        
        
    con.commit()
    print ('Sensor to sensor distances calculated' )

        
SensorDistances(con,cur)




computing lengths ..
                                odrouteid  originsiteid     midsiteid  \
0  MAC000010102>MAC000010118>MAC000010104  MAC000010102  MAC000010118   
1  MAC000010102>MAC000010119>MAC000010104  MAC000010102  MAC000010119   
2  MAC000010101>MAC000010104>MAC000010119  MAC000010101  MAC000010104   
3  MAC000010101>MAC000010118>MAC000010119  MAC000010101  MAC000010118   
4  MAC000010123>MAC000010112>MAC000010120  MAC000010123  MAC000010112   
5  MAC000010123>MAC000010114>MAC000010120  MAC000010123  MAC000010114   
6  MAC000010121>MAC000010113>MAC000010124  MAC000010121  MAC000010113   
7  MAC000010121>MAC000010112>MAC000010124  MAC000010121  MAC000010112   
8  MAC000010119>MAC000010109>MAC000010130  MAC000010119  MAC000010109   
9  MAC000010119>MAC000010104>MAC000010130  MAC000010119  MAC000010104   

     destsiteid        ox        oy        mx        my        dx        dy  
0  MAC000010104  435621.0  370547.0  436617.0  370741.0  438388.0  369743.0  
1  MAC000010104  43

Shortest path between mid and destination sensors found
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010102>MAC000010119>MAC000010104', 834, 12);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010102>MAC000010119>MAC000010104', 835, 26);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010102>MAC000010119>MAC000010104', 836, 30);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010102>MAC000010119>MAC000010104', 2229, 6);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010102>MAC000010119>MAC000010104', 2257, 10);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010102>MAC000010119>MAC000010104', 2258, 32);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010102>MAC000010119>MAC000010104', 3047, 114);
INSERT INTO Sens

Mid GIDs selected
Dest GIDs selected
Shortest path between origin and mid sensors found
Shortest path between mid and destination sensors found
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010101>MAC000010104>MAC000010119', 85, 27);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010101>MAC000010104>MAC000010119', 86, 21);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010101>MAC000010104>MAC000010119', 2654, 12);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010101>MAC000010104>MAC000010119', 2655, 6);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010101>MAC000010104>MAC000010119', 2656, 68);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010101>MAC000010104>MAC000010119', 3003, 162);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)   

Shortest path between mid and destination sensors found
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010101>MAC000010118>MAC000010119', 117, 23);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010101>MAC000010118>MAC000010119', 118, 23);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010101>MAC000010118>MAC000010119', 119, 12);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010101>MAC000010118>MAC000010119', 120, 11);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010101>MAC000010118>MAC000010119', 221, 41);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010101>MAC000010118>MAC000010119', 906, 11);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010101>MAC000010118>MAC000010119', 1356, 48);
INSERT INTO SensorD

INSERT INTO SensorDistances(ODrouteID, gid_link, md_length)             VALUES ('MAC000010101>MAC000010118>MAC000010119', 8062, 25);
INSERT INTO SensorDistances(ODrouteID, gid_link, md_length)             VALUES ('MAC000010101>MAC000010118>MAC000010119', 8063, 155);
INSERT INTO SensorDistances(ODrouteID, gid_link, md_length)             VALUES ('MAC000010101>MAC000010118>MAC000010119', 8286, 352);
INSERT INTO SensorDistances(ODrouteID, gid_link, md_length)             VALUES ('MAC000010101>MAC000010118>MAC000010119', 8290, 41);
INSERT INTO SensorDistances(ODrouteID, gid_link, md_length)             VALUES ('MAC000010101>MAC000010118>MAC000010119', 14600, 5);
INSERT INTO SensorDistances(ODrouteID, gid_link, md_length)             VALUES ('MAC000010101>MAC000010118>MAC000010119', 14601, 20);
INSERT INTO SensorDistances(ODrouteID, gid_link, md_length)             VALUES ('MAC000010101>MAC000010118>MAC000010119', 14602, 13);
INSERT INTO SensorDistances(ODrouteID, gid_link, md_length)      

Mid GIDs selected
Dest GIDs selected
Shortest path between origin and mid sensors found
Shortest path between mid and destination sensors found
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010123>MAC000010114>MAC000010120', 117, 23);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010123>MAC000010114>MAC000010120', 118, 23);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010123>MAC000010114>MAC000010120', 119, 12);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010123>MAC000010114>MAC000010120', 120, 11);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010123>MAC000010114>MAC000010120', 12098, 9);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010123>MAC000010114>MAC000010120', 12099, 9);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)   

Shortest path between mid and destination sensors found
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010121>MAC000010112>MAC000010124', 5727, 12);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010121>MAC000010112>MAC000010124', 5829, 7);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010121>MAC000010112>MAC000010124', 5846, 13);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010121>MAC000010112>MAC000010124', 5848, 31);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010121>MAC000010112>MAC000010124', 6185, 9);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010121>MAC000010112>MAC000010124', 6186, 36);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010121>MAC000010112>MAC000010124', 6187, 38);
INSERT INTO Sen

Shortest path between mid and destination sensors found
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010119>MAC000010109>MAC000010130', 583, 109);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010119>MAC000010109>MAC000010130', 584, 83);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010119>MAC000010109>MAC000010130', 1356, 48);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010119>MAC000010109>MAC000010130', 1357, 28);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010119>MAC000010109>MAC000010130', 2187, 86);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010119>MAC000010109>MAC000010130', 8059, 175);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010119>MAC000010109>MAC000010130', 8286, 352);
INSERT INTO 

Shortest path between mid and destination sensors found
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010119>MAC000010104>MAC000010130', 8287, 27);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010119>MAC000010104>MAC000010130', 8493, 75);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010119>MAC000010104>MAC000010130', 8494, 84);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010119>MAC000010104>MAC000010130', 8495, 35);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010119>MAC000010104>MAC000010130', 8496, 39);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010119>MAC000010104>MAC000010130', 8497, 64);
INSERT INTO SensorDistances(ODrouteID, gid_link, om_length)             VALUES ('MAC000010119>MAC000010104>MAC000010130', 14076, 4);
INSERT INTO S

In [81]:
def createTotalRouteCount(con, cur):
    sql = "DROP TABLE IF EXISTS Totalroutecount;"
    cur.execute(sql)          
    sql="CREATE TABLE Totalroutecount ( \
            emp_no SERIAL PRIMARY KEY, \
            ODrouteID text, \
            timestamp timestamp, \
            winlenseconds integer, \
            count integer \
            );"
    cur.execute(sql)          
    con.commit()
createTotalRouteCount(con, cur)

def TotalrouteCounts(con,cur):   #count number of matching Bluetooth detections between origins and destinations
    sql = "SELECT * FROM TotalRoute;"
    df_odroute = pd.read_sql_query(sql,con)
    print (df_odroute)
    for i in range(0, df_odroute.shape[0]):    #each route
        oSiteID = df_odroute['originsiteid'][i]
        #mSiteID = df_odroute['midsiteid'][i]
        dSiteID = df_odroute['destsiteid'][i]
        #MAC matching
        sql = "SELECT d.siteID AS dSiteID,  d.mac as dmac, \
        d.timestamp as dtimestamp, o.siteID AS oSiteID,  \
        o.mac as omac, o.timestamp as otimestamp    \
        FROM Detection_clean AS d, Detection_clean AS o  \
        WHERE d.timestamp>o.timestamp\
        AND o.mac=d.mac AND o.siteID='%s'\
        AND d.siteID='%s';"%(oSiteID, dSiteID)
        print(sql)
        df_matches = pd.read_sql_query(sql,con)
        count = df_matches.shape[0]  #count number of bluetooth matches
        #these two variables allow us to compute flows for differnt time windows. Here we just take one whole day.
        winlenseconds = 99999999.9
        timestamp = "2015-02-14 09:00:00"
        sql = "INSERT INTO TotalRouteCount (ODrouteID, timestamp,\
        winlenseconds, count) VALUES ('%s', '%s', %f, %i)"\
        %(df_odroute['odrouteid'].iloc[i], df_odroute['timestamp'].iloc[i], winlenseconds, count)
        cur.execute(sql)
        print (sql)
        con.commit()
TotalrouteCounts(con,cur)


def createODRouteCount(con, cur):
    sql = "DROP TABLE IF EXISTS ODroutecount;"
    cur.execute(sql)          
    sql="CREATE TABLE ODroutecount ( \
            emp_no SERIAL PRIMARY KEY, \
            ODrouteID text, \
            timestamp timestamp, \
            winlenseconds integer, \
            count integer \
            );"
    cur.execute(sql)          
    con.commit()
createODRouteCount(con, cur)

def ODrouteCounts(con,cur):   #count number of matching Bluetooth detections between origins and destinations
    sql = "SELECT * FROM ODRoute;"
    df_odroute = pd.read_sql_query(sql,con)
    print (df_odroute)
    for i in range(0, df_odroute.shape[0]):    #each route
        oSiteID = df_odroute['originsiteid'][i]
        mSiteID = df_odroute['midsiteid'][i]
        dSiteID = df_odroute['destsiteid'][i]
        #MAC matching
        sql = "SELECT d.siteID AS dSiteID,  d.mac as dmac, \
        d.timestamp as dtimestamp  , m.siteID AS mSiteID,  m.mac as mmac, \
        m.timestamp as mtimestamp,  o.siteID AS oSiteID,  \
        o.mac as omac, o.timestamp as otimestamp    \
        FROM Detection_clean AS d, Detection_clean AS m, Detection_clean AS o  \
        WHERE d.timestamp>m.timestamp AND m.timestamp>o.timestamp\
        AND o.mac=m.mac AND m.mac=d.mac AND o.siteID='%s'\
        AND m.siteID = '%s' AND d.siteID='%s';"%(oSiteID, mSiteID, dSiteID)
        print(sql)
        df_matches = pd.read_sql_query(sql,con)
        count = df_matches.shape[0]  #count number of bluetooth matches
        #these two variables allow us to compute flows for differnt time windows. Here we just take one whole day.
        winlenseconds = 99999999.9
        timestamp = "2015-02-14 09:00:00"
        sql = "INSERT INTO ODRouteCount (ODrouteID, timestamp,\
        winlenseconds, count) VALUES ('%s', '%s', %f, %i)"\
        %(df_odroute['odrouteid'].iloc[i], df_odroute['timestamp'].iloc[i], winlenseconds, count)
        cur.execute(sql)
        print (sql)
        con.commit()
ODrouteCounts(con,cur)


   emp_no                  odrouteid           timestamp  winlenseconds  \
0       1  MAC000010119>MAC000010130 2015-02-14 09:00:00      100000000   
1       2  MAC000010102>MAC000010104 2015-02-14 09:00:00      100000000   
2       3  MAC000010121>MAC000010124 2015-02-14 09:00:00      100000000   
3       4  MAC000010101>MAC000010119 2015-02-14 09:00:00      100000000   
4       5  MAC000010123>MAC000010120 2015-02-14 09:00:00      100000000   

   count  originsiteid    destsiteid  
0   2555  MAC000010119  MAC000010130  
1   1020  MAC000010102  MAC000010104  
2      4  MAC000010121  MAC000010124  
3    585  MAC000010101  MAC000010119  
4     63  MAC000010123  MAC000010120  
SELECT d.siteID AS dSiteID,  d.mac as dmac,         d.timestamp as dtimestamp, o.siteID AS oSiteID,          o.mac as omac, o.timestamp as otimestamp            FROM Detection_clean AS d, Detection_clean AS o          WHERE d.timestamp>o.timestamp        AND o.mac=d.mac AND o.siteID='MAC000010119'        AND d.sit

INSERT INTO ODRouteCount (ODrouteID, timestamp,        winlenseconds, count) VALUES ('MAC000010121>MAC000010112>MAC000010124', '2015-02-14 09:00:00', 99999999.900000, 0)
SELECT d.siteID AS dSiteID,  d.mac as dmac,         d.timestamp as dtimestamp  , m.siteID AS mSiteID,  m.mac as mmac,         m.timestamp as mtimestamp,  o.siteID AS oSiteID,          o.mac as omac, o.timestamp as otimestamp            FROM Detection_clean AS d, Detection_clean AS m, Detection_clean AS o          WHERE d.timestamp>m.timestamp AND m.timestamp>o.timestamp        AND o.mac=m.mac AND m.mac=d.mac AND o.siteID='MAC000010101'        AND m.siteID = 'MAC000010104' AND d.siteID='MAC000010119';
INSERT INTO ODRouteCount (ODrouteID, timestamp,        winlenseconds, count) VALUES ('MAC000010101>MAC000010104>MAC000010119', '2015-02-14 09:00:00', 99999999.900000, 737)
SELECT d.siteID AS dSiteID,  d.mac as dmac,         d.timestamp as dtimestamp  , m.siteID AS mSiteID,  m.mac as mmac,         m.timestamp as mtimestamp,

In [92]:
def createODLinkCount(con, cur):
    sql = "DROP TABLE IF EXISTS ODlinkcount;"
    cur.execute(sql)          
    sql="CREATE TABLE ODlinkcount ( \
            emp_no SERIAL PRIMARY KEY, \
            gid text, \
            timestamp timestamp, \
            winlenseconds integer, \
            count integer \
            );"
    cur.execute(sql)          
    con.commit()
createODLinkCount(con, cur)

def ODlinkCounts(con, cur):
    sql = "SELECT * FROM ODRouteCount;"
    df_rc = pd.read_sql_query(sql,con)
    for i in range(0, df_rc.shape[0]):    #each routecount
        row  = df_rc.iloc[i]
        count         = row['count']
        winlenseconds = row['winlenseconds']
        timestamp     = row['timestamp']
        if count>0:
            sql = "SELECT * FROM SensorDistances WHERE ODrouteid='%s'"%row['odrouteid']
            df_rl = pd.read_sql_query(sql,con)
            for j in range(0, df_rl.shape[0]): #add counts for each link
                link = df_rl.iloc[j]                   #route component
                gid_link = link['gid_link']
                sql = "INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,\
                count) VALUES ('%s','%s', %i, %i);"%(gid_link, \
                timestamp, winlenseconds, count)
                print(sql)
                cur.execute(sql)
            con.commit()
ODlinkCounts(con, cur)


INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('8287','2015-02-14 09:00:00', 100000000, 3887);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('8493','2015-02-14 09:00:00', 100000000, 3887);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('8494','2015-02-14 09:00:00', 100000000, 3887);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('8495','2015-02-14 09:00:00', 100000000, 3887);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('8496','2015-02-14 09:00:00', 100000000, 3887);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('8497','2015-02-14 09:00:00', 100000000, 3887);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('14076','2015-02-14 09:00:00', 100000000, 3887);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,      

INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('14925','2015-02-14 09:00:00', 100000000, 742);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('14926','2015-02-14 09:00:00', 100000000, 742);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('14929','2015-02-14 09:00:00', 100000000, 742);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('14932','2015-02-14 09:00:00', 100000000, 742);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('14939','2015-02-14 09:00:00', 100000000, 742);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('14940','2015-02-14 09:00:00', 100000000, 742);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('14941','2015-02-14 09:00:00', 100000000, 742);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,       

INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('9','2015-02-14 09:00:00', 100000000, 12);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('678','2015-02-14 09:00:00', 100000000, 12);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('2717','2015-02-14 09:00:00', 100000000, 12);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('5847','2015-02-14 09:00:00', 100000000, 12);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('12804','2015-02-14 09:00:00', 100000000, 12);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('13571','2015-02-14 09:00:00', 100000000, 12);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('14154','2015-02-14 09:00:00', 100000000, 12);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count)

INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('8495','2015-02-14 09:00:00', 100000000, 737);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('8496','2015-02-14 09:00:00', 100000000, 737);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('8497','2015-02-14 09:00:00', 100000000, 737);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('14076','2015-02-14 09:00:00', 100000000, 737);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('14077','2015-02-14 09:00:00', 100000000, 737);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('14080','2015-02-14 09:00:00', 100000000, 737);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('14383','2015-02-14 09:00:00', 100000000, 737);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,          

INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('117','2015-02-14 09:00:00', 100000000, 100);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('118','2015-02-14 09:00:00', 100000000, 100);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('119','2015-02-14 09:00:00', 100000000, 100);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('120','2015-02-14 09:00:00', 100000000, 100);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('12098','2015-02-14 09:00:00', 100000000, 100);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('12099','2015-02-14 09:00:00', 100000000, 100);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,                count) VALUES ('12301','2015-02-14 09:00:00', 100000000, 100);
INSERT INTO ODlinkcount (gid, timestamp, winlenseconds,               

In [94]:
def plotFlows(con,cur):     
    dt_start  = datetime.datetime.strptime('2015-01-05_00:00:00' , \
                                           "%Y-%m-%d_%H:%M:%S" )
    dt_end    = datetime.datetime.strptime('2016-12-10_00:00:00' , \
                                           "%Y-%m-%d_%H:%M:%S" )
    sql = "SELECT ways.gid, SUM(ODlinkcount.count), ways.the_geom FROM ways,\
    ODlinkcount  WHERE ODlinkcount.gid::int=ways.gid AND ODlinkcount.timestamp>'%s'\
    AND ODlinkcount.timestamp<'%s'  GROUP BY ways.gid;"%(dt_start,dt_end)
    print(sql) 
    df_links = gpd.GeoDataFrame.from_postgis(sql,con,geom_col='the_geom')
    for i in range(0,df_links.shape[0]): 
        link = df_links.iloc[i]
        lons = link['the_geom'].coords.xy[0] #coordinates in latlon
        lats = link['the_geom'].coords.xy[1]
        gid = int(link.gid) 
        xs=[];ys=[]
        n_segments = len(lons)
        for j in range(0, n_segments):
            (x,y) = pyproj.transform(wgs84, bng, lons[j], lats[j]) #project to BNG -- uses nonISO lonlat convention  #TODO faster to cache this! 
            xs.append(x)
            ys.append(y)
        color='r'
        lw = int(link['sum']/10000)
        plot(xs, ys, color, linewidth=lw)  
plotFlows(con,cur)
print ("Flows plotted")


SELECT ways.gid, SUM(ODlinkcount.count), ways.the_geom FROM ways,    ODlinkcount  WHERE ODlinkcount.gid::int=ways.gid AND ODlinkcount.timestamp>'2015-01-05 00:00:00'    AND ODlinkcount.timestamp<'2016-12-10 00:00:00'  GROUP BY ways.gid;
Flows plotted


In [ ]:
#plot of Flows vs route id

In [ ]:
#plot of Flows vs od distances

In [ ]:
#Difference from Shortest path

In [108]:
import plotly.plotly as py
import plotly.graph_objs as go

import pandas as pd

mapbox_access_token = pk.eyJ1Ijoic3JlZXNvbWFkYXMiLCJhIjoiY2poZXY5aDNoMDNjNDM3bnJhaXZneHF5YyJ9.5io2Q2NCWyJTE82MSK5PdA


df = pd.read_csv("/headless/data/BTsites.csv")
print (df)
site_lat = df.Latitude
site_lon = df.Longitude
locations_name = df.Description

data = [
    go.Scattermapbox(
        lat=site_lat,
        lon=site_lon,
        mode='markers',
        marker=dict(
            size=17,
            color='rgb(255, 0, 0)',
            opacity=0.7
        ),
        text=locations_name,
        hoverinfo='text'
    ),
    go.Scattermapbox(
        lat=site_lat,
        lon=site_lon,
        mode='markers',
        marker=dict(
            size=8,
            color='rgb(242, 177, 172)',
            opacity=0.7
        ),
        hoverinfo='none'
    )]


layout = go.Layout(
    title='Bluetooth Sites',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=38,
            lon=-94
        ),
        pitch=0,
        zoom=3,
        style='light'
    ),
)

fig = dict(data=data, layout=layout)

py.iplot(fig, filename='Bluetooth Sites')


SyntaxError: invalid syntax (<ipython-input-108-bdcc4127f27a>, line 6)